In [ ]:
test01 = Generic(
    "TEST01"
    ,{"collect":["2023-09-01"   ,"2023-10-01"] # next pool collection date # last pool collection date,
    ,"pay":["2023-09-15"  ,"2023-10-15"] # last distribution payment date, # next distribution date
    ,"stated":"2025-09-28",
    "firstPay":"2020-11-15",
    "Settle Date":"2023-09-28",
    "poolFreq":"MonthEnd"
    ,"payFreq":["DayOfMonth",15]
    }
    ,{'assets':[
        ["Mortgage"
        ,{"originBalance":10_000_000,"originRate":["fix",0.08],"originTerm":60
          ,"freq":"Monthly","type":"Level","originDate":"2020-09-28"}
          ,{"currentBalance":5_000_000
          ,"currentRate":0.09
          ,"remainTerm":48
          ,"borrowerNum":1
          ,"status":"current"}]]}
    ,(("collections",{"balance":500_000}),("revolBuyAcc",{"balance":0}),("cashReserve",{"balance":550_000,"type":{"targetReserve":[("poolBalance",),0.10]}}))
    ,(("A1",{"balance":4_000_000
             ,"rate":0.10
             ,"originBalance":10_000_000
             ,"originRate":0.10
             ,"startDate":"2020-09-28"
             ,"rateType":{"Fixed":0.10}
             ,"maturityDate":"2025-09-28"
             ,"bondType":{"Lockout":"2024-03-28"}})
      ,("B",{"balance":1
             ,"rate":0.0
             ,"originBalance":100
             ,"originRate":0.00
             ,"startDate":"2020-09-28"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,
    (("servicer_fee",{"type":{"annualPctFee":[("poolBalance",),0.02]}}),)
    ,{"default":[
       ["transfer",'revolBuyAcc',"collections"]
         ,["transfer","cashReserve","collections"]
         ,["payInt","collections",["A1"]]
         ,["payFee",["collections"],['servicer_fee']]      
         ,["IfElse" ,["status","Revolving"] # acitons in the revolving period
           ,[["transferBy",{"formula":("substract",("bondBalance",),("poolBalance",))}
                          ,"collections",'revolBuyAcc']
            ,["buyAsset",["Current|Defaulted",1.0,0],"revolBuyAcc",None] # buy asset with 1:1 if asset with performing
            ,["payResidual","collections",["B"]]]
           ,[["payPrin","collections",["A1"]] # actions if deal is in Amortizing status
            ,["payPrin","collections",["B"]]
            ,["payResidual", "collections", ["B"]]
            ]]
         ]
     ,"endOfCollection":[["calcFee","servicer_fee"]] # accure fee by end of collection period
     ,"cleanUp":[["sellAsset", ["Current|Defaulted",1.0,0], "collections"]
                 ,["accrueAndPayInt","collections",["A"]]
                 ,["payPrin","collections",["A1"]]]
     }
         
    ,[["CollectedInterest","collections"]
      ,["CollectedPrincipal","collections"]
      ,["CollectedPrepayment","collections"]
      ,["CollectedRecoveries","collections"]]
    ,None
    ,None
    ,None
    ,
    {"BeforeDistribution":[
        {"condition":["any"
                      ,[">=","2024-03-28"]]
        ,"effects":("newStatus","Amortizing")
        ,"status":False
        ,"curable":False}]     
     }
    ,"Revolving")


revol_asset = ["Mortgage"
                ,{"originBalance":100_000,"originRate":["fix",0.20],"originTerm":24
                  ,"freq":"Monthly","type":"Level","originDate":"2023-09-01"}
                  ,{"currentBalance":100_000
                  ,"currentRate":0.20
                  ,"remainTerm":24
                  ,"status":"current"}]

r = localAPI.run(test01,
                 assumptions=[{"RevolvingAssets":[["constant",[revol_asset]]  # revolving pool can be a list of assets
                                            ,[{"CDR":0.01}]]}, 
                   {"CPR":[0.1]}
                             ,{"CDR":0.07}
                             ,{"Recovery":(0.1,12)}],
                 #pricing={"PVDate":"2023-06-22"
                 #        ,"PVCurve":[["2020-01-01",0.025]]},
                 read=True)
print(r)